# Vector Similarity Astra-Bedrock Search QA Quickstart

Set up a simple Question-Answering system with LangChain and AWS Bedrock, using Cassandra as the Vector Database.

Make sure you have a Database and get ready to upload the Secure Connect Bundle and supply the Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details. Remember you need a **custom Token** with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).



### Set up your python environment




In [1]:
!pwd
!curl -H 'Cache-Control: no-cache, no-store'  https://raw.githubusercontent.com/synedra/astra_vector_examples/main/notebook/requirements_bedrock.txt --output requirements.txt
%pip install -r requirements.txt


### Import needed libraries



In [2]:
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT, ProtocolVersion
from cassandra.auth import PlainTextAuthProvider
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.vectorstores.cassandra import Cassandra
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
import boto3, json, os, sys
from getpass import getpass
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader


### Astra Setup



### Astra DB Secure Connect Bundle

Please upload the Secure Connect Bundle zipfile to connect to your Astra DB instance.

The Secure Connect Bundle is needed to establish a secure connection to the database.
Click [here](https://awesome-astra.github.io/docs/pages/astra/download-scb/#c-procedure) for instructions on how to download it from Astra DB.

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name (e.g. vector): ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
from getpass import getpass
ASTRA_DB_APPLICATION_TOKEN = getpass('Your Astra DB Token ("AstraCS:..."): ')

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider
profile = ExecutionProfile(request_timeout=30)


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                "token",
                ASTRA_DB_APPLICATION_TOKEN,
            ),
           protocol_version=ProtocolVersion.V4,
           execution_profiles={EXEC_PROFILE_DEFAULT: profile},
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unsupported CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unsupported CQL Session mode')

In [2]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

### AWS Credential Setup



In [ ]:
# Input your AWS Access Key ID
os.environ["AWS_ACCESS_KEY_ID"] = getpass('Your AWS Access Key ID: ')

In [ ]:
# Input your AWS Secret Access Key
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass('Your AWS Secret Access Key: ')

In [ ]:
# Input your AWS Session Token
os.environ["AWS_SESSION_TOKEN"] = getpass('Your AWS Session Token: ')

### Set up AWS Bedrock objects



In [3]:
bedrock_runtime=boto3.client('bedrock-runtime','us-west-2',endpoint_url='https://bedrock.us-west-2.amazonaws.com')
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_runtime)


### Set up a vectorstore in Astra

In [4]:
myCassandraVStore = Cassandra(
embedding=bedrock_embeddings,
session=session,
keyspace=ASTRA_DB_KEYSPACE,
table_name='shakespeare_act5'
)


### Populate the database

Add lines for the text of Romeo and Astra, Scene 5, Act 3



In [ ]:
# retrieve the text of a scene from act 5 of Romeo and Astra. 
# Juliet's name was changed to Astra to prevent the LLM from "cheating" when providing an answer.
! mkdir texts
! curl https://raw.githubusercontent.com/synedra/astra_vector_examples/main/romeo_astra.json --output texts/romeo_astra.json
quote_array = json.load(open('texts/romeo_astra.json'))


Next, you'll populate the database with the lines from the play.
This can take a couple of minutes, please be patient.  In total there are 321 lines.


In [5]:
for index in range(len(quote_array)):
 location = ''
 if (quote_array[index]['ActSceneLine'] != ''):
    (act, scene, line) = quote_array[index]['ActSceneLine'].split('.')
    location = 'Act {}, Scene {}, Line {}'.format(act, scene, line)
 quote_input = '{} : {} : {}'.format(location, quote_array[index]['Player'], quote_array[index]['PlayerLine'])
 input_document = Document(page_content=quote_input)
 print(quote_input)
 myCassandraVStore.add_documents(documents=[input_document])


### Ask a question
Enter a question and create the prompt template for the LLM query.  
Give instructions about what you want the model to do.


In [ ]:
# Enter a question about Romeo and Astra (Like 'How did Astra die?')
QUESTION_FOR_MODEL = input('Your question about Romeo and Astra: ')

In [6]:
generation_prompt_template = """Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""


### Create a retriever
This will allow you to get the most relevant documents from the database


In [6]:
retriever = myCassandraVStore.as_retriever(search_kwargs={'k':2,})
output=retriever.get_relevant_documents(QUESTION_FOR_MODEL)
prompt = PromptTemplate.from_template("{page_content}")
context = ''
for document in output:
    context += '\n ' + document.page_content 

print(context)


### Build the llm prompt


In [6]:
llm_prompt = generation_prompt_template.format(
    question=QUESTION_FOR_MODEL,
    context=context,
)


### Generate the Answer 
Generate the answer using the prompt, using amazon.titan-tg1-large as the LLM model


In [6]:
body = json.dumps({"prompt": llm_prompt, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2"
accept = 'application/json'
contentType = 'application/json'
response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
print ('Question: ' + QUESTION_FOR_MODEL)
print ('Answer: ' + response_body.get('completion'))


### Additional resources
To learn more about Amazon Bedrock you can check out this site:  https://github.com/aws-samples/amazon-bedrock-samples/tree/main/introduction-to-bedrock